# Intro *machine learning*

In [1]:
import pandas as pd

## *Supervised learning*

![](figures/iris.png)

#### Opdracht

In de file `data/iris/Iris.csv` zit data over irissen.

- Open deze data in een text editor en als een werkblad (Excel of soortgelijk).  Wat zie je?
- Laad de iris dataset in in een pandas dataframe met `iris = pd.read_csv('../data/iris/Iris.csv', delimiter=',', header=0)
`.  Print de dataframe.
- Uit hoeveel individuele irissen bestaat de dataset?
- Welke kolommen bevatten meetwaarden?
- In welke kolom zit de soortnaam?

In [2]:
# Oplossing

iris = pd.read_csv('../data/iris/Iris.csv', delimiter=',', header=0)
iris

Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm  \
0      1            5.1           3.5            1.4           0.2   
1      2            4.9           3.0            1.4           0.2   
2      3            4.7           3.2            1.3           0.2   
3      4            4.6           3.1            1.5           0.2   
4      5            5.0           3.6            1.4           0.2   
..   ...            ...           ...            ...           ...   
145  146            6.7           3.0            5.2           2.3   
146  147            6.3           2.5            5.0           1.9   
147  148            6.5           3.0            5.2           2.0   
148  149            6.2           3.4            5.4           2.3   
149  150            5.9           3.0            5.1           1.8   

            Species  
0       Iris-setosa  
1       Iris-setosa  
2       Iris-setosa  
3       Iris-setosa  
4       Iris-setosa  
..              ...  
145  Iris-virginica  
146  Iris-virginica  
147  Iris-virginica  
148  Iris-virginica  
149  Iris-virginica  

[150 rows x 6 columns]

### Een beetje jargon
- *Features*: hetgeen we altijd weten, en wat we gebruiken als input voor het model.  Hier zijn dat de gemeten kenmerken van de bloemen: `SepalLengthCm`, `SepalWidthCm`, `PetalLengthCm` en `PetalWidthCm`.
- *Target*: hetgeen we willen voorspellen met ons model.  Hier is dat de soort van iris: `Species`.

We willen voorspellen tot welke soort een individuele iris behoort, en we hebben daarbij de keuze uit drie klassen: `Iris-setosa`, `Iris-versicolor`, en `Iris-virginica`.  Dat betekent dat we hier spreken van een klassificatieprobleem.  Er bestaan nog andere problemen in *machine learning*.  Veelvoorkomend is regressie, waarbij we een of meerdere re\"ele getallen proberen te voorspellen.  In computervisie (zie verder) kunnen we bijvoorbeeld bepaalde objecten proberen te detecteren op foto's.

Er bestaan enorm veel modellen binnen *machine learning*.  Ze hebben meestal als doel om op basis van gekende data iets nieuws te voorspellen.  In dit geval hebben we een aantal voorbeelden waarvan zowel de inputs van het model (*features*) als de outputs (*targets*) gekend zijn.  Stel dat wij als niet-biologen, dus als mensen die de iris niet kunnen determineren (klassificeren), nu een nieuwe iris meten.  Een goed model, gebouwd op basis van data die wel door biologen is opgesteld, kan ons dan helpen om de nieuwe iris toch te determineren.

Typisch gaat men de dataset, waarvan we iets willen leren eerst splitsen in een train set en test set.  De reden daarvoor wordt zo dadelijk duidelijk.  We willen dat beide sets van elke klasse ongeveer evenveel voorbeelden hebben.  Verder willen we dat 75% van de volledige dataset naar de train set en 25% naar de test set gaat.

Onderstaande code splitst de dataset willekeurig op in een train en test set.

In [3]:
# Shuffle
iris = iris.sample(frac=1).reset_index(drop=True)

# Split
n_train = len(iris)*3//4
iris_train = iris[:n_train]
iris_test = iris[n_train:]

#### Opdracht

- Wat gebeurt er in de eerste regel code: `iris = iris.sample(frac=1).reset_index(drop=True)`?
- Waarom is dat nodig?

### Een heel simpel model

Stel dat we enkel naar de eerste *feature*-kolom `SepalLengthCm` van de train set kijken.  We noemen deze variabele voorlopig $x$. Op basis daarvan kunnen we een heel simpel model maken:

- We bereken voor elke klasse de gemiddelde waarde van $x$
- We ordenen de klasses volgens deze gemiddeldes
- We berekenen de twee middelpunten $m_0$ en $m_1$ tussen de gemiddeldes
- Het model werkt dan als volgt:
    - Elke iris waarvoor $x <= m_0$ voorspellen we als horende bij de eerste klasse
    - Elke iris waarvoor $m_0 < x <= m_1$ voorspellen we als horende bij de eerste klasse
    - Elke iris waarvoor $x > m_1$ voorspellen we als horende bij de eerste klasse

Onderstaande code berekent eerst de klassegemiddeldes op basis van de train set en daarna $m_0$ en $m_1$.


In [4]:
class_means = iris[['SepalLengthCm', 'Species']].groupby('Species').aggregate('mean').sort_values('SepalLengthCm')
print(class_means)
print()

m_0 = (class_means['SepalLengthCm'].iloc[0] + class_means['SepalLengthCm'].iloc[1]) / 2
m_1 = (class_means['SepalLengthCm'].iloc[1] + class_means['SepalLengthCm'].iloc[2]) / 2

print(m_0, m_1)


                 SepalLengthCm
Species                       
Iris-setosa              5.006
Iris-versicolor          5.936
Iris-virginica           6.588

5.471 6.262


#### Opdracht

- Schrijf een functie `iris_classifier` die de klassificatie uitvoert.  Deze functie neemt een getal `x` als input en geeft de voorspelde klasse als output (een string).
- Test de functie op enkele willekeurige waarden voor $x$.

In [5]:
# Oplossing

def iris_classifier(x):
    if x <= m_0:
        return 'Iris-setosa'
    elif x > m_1:
        return 'Iris-virginica'
    else:
        return 'Iris-versicolor'

print(iris_classifier(x = 6))

Iris-versicolor


Proficiat! Je hebt nu een model getraind op data.  Dit model is extreem simpel, maar de workflow is voor zowat alle gesuperviseerde klassificatiemodellen dezelfde.

### Evaluatie

Zoals gezegd, bestaan er enorm veel modellen.  Het is uiteraard de bedoeling om een zo goed mogelijk model te maken.  Onze simpele `iris_classifier` heeft twee parameters: $m_0$ en $m_1$.  We hebben gekozen om deze gelijk te nemen aan de middelpunten van de klassegemiddeldes van de *feature* $x$.  Misschien wordt het model beter als we de parameters lichtjes aanpassen?

Maar wanneer is het ene model eigenlijk beter dan het andere?  In het geval van klassificatie er een gemakkelijk antwoord op die vraag.  Het model dat voor de meeste instanties een juiste voorspelling doet.

#### Opdracht

- Voorspel voor alle irissen uit de test set de klasse door de waarde van `SepalLengthCm` in te geven in `iris_classifier`.
- Tel het aantal keren dat deze voorspelde klasse dezelfde is als de werkelijke klasse.
- Zet om naar een score in procent.  Dit is de *accuraatheid* van het model voor de test set.

In [6]:
# Oplossing

x_test = list(iris_test['SepalLengthCm'])
y_test = list(iris_test['Species'])

count_correct = 0
for i in range(len(x_test)):
    pred = iris_classifier(x_test[i])
    if pred == y_test[i]:
        count_correct += 1

print(count_correct)
print(100*count_correct/len(x_test), '%')

27
71.05263157894737 %


#### Extra opdrachten

- Voorspel de klassen van de train set en evalueer.  Waarom is de score lager/hoger?
- Verschuif de waarden van de modelparameters $m_0$ en $m_1$ lichtjes.  Kan je een beter model maken?
- Gebruik een andere kolom dan `SepalLengthCm` om je model te maken.  Werkt dit beter?  Welke kolom is het meest informatief?
- Maak een complexer model dat rekent houdt met meerdere *feature*-kolommen.